# Pipeline for BIAPT lab EEG Preprocessing: 
#### inspired by: https://github.com/hoechenberger/pybrain_mne/
#### adapted by: Charlotte Maschke
#### This pipeline uses MNE Python to preprocess EEG data: Plese go here: 
####                                https://github.com/hoechenberger/pybrain_mne/ 
####  for more documentation on MNE Python

## Some setup and import

In [1]:
import matplotlib
#import pathlib
import mne

# Ensure Matplotlib uses the Qt5Agg backend, 
# which is the best choice for MNE-Python's 
# interactive plotting functions.
matplotlib.use('Qt5Agg')

### Which subject do you want to preprocess? 

In [2]:
ID = "sub-xxYY"
task = "task-Rest"

In [3]:
raw_path = "../BIDS_testdata/source/{}/eeg/{}_{}.mff".format(ID,ID,task)
raw_path

'../BIDS_testdata/source/sub-xxYY/eeg/sub-xxYY_task-Rest.mff'

## Load the raw data!

In [4]:
raw = mne.io.read_raw_egi(raw_path)
raw

Reading EGI MFF Header from C:\Users\User\Documents\GitHub\EEG_Preprocessing\BIDS_testdata\source\sub-xxYY\eeg\sub-xxYY_task-Rest.mff...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {} ...


<RawMff | signal1.bin, 166 x 2245322 (2245.3 s), ~213 kB, data not loaded>

## Crop the data

crop on 2 ends:  .crop(tmin = 000 , tmax = 000)  
crop beginning:  .crop(tmin = 100 )  
crop end only :  .crop(tmax = 100)

#### the values here are in seconds ! 

In [5]:
# show the duration of your signal
print("Length of Signal in min: ")
raw.times[-1]/60

Length of Signal in min: 


37.422016666666664

In [6]:
# this can be used to transform minutes to milliseconds
ms_min = 1  * 60 
ms_max = 3  * 60

In [7]:
# input here your minimal and maximal time you want to keep 
raw_cropped = raw.copy().crop(tmin = ms_min, tmax = ms_max)

## If you have any events, extract them

In [8]:
events = mne.find_events(raw_cropped)

In [9]:
len(events)

0

### Keep only the EEG

In [10]:
# this is to load EEG. If you want to load other stuff please refer to the website documetation
eeg_cropped = raw_cropped.pick_types(eeg = True)
print('Number of channels in EEG:')
len(eeg_cropped.ch_names)

Number of channels in EEG:


129

## Filter the data

In [11]:
# load actual data into system (before it was only metadata)
eeg_cropped.load_data()

# filter the data between 0.1 to 55 Hz
eeg_cropped_filtered = eeg_cropped.filter(l_freq=0.1, h_freq=55)

Reading 0 ... 120000  =      0.000 ...   120.000 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 55 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 55.00 Hz
- Upper transition bandwidth: 13.75 Hz (-6 dB cutoff frequency: 61.88 Hz)
- Filter length: 33001 samples (33.001 sec)



## Let's visualize the data!

In [14]:
eeg_cropped_filtered.plot(events=events)

<MNEBrowseFigure size 1280x960 with 4 Axes>

Channels marked as bad: ['E111', 'E118', 'E62', 'E55']


# _____________ This is where I stopped for now ________________________________

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2)

raw_eeg_cropped.plot_psd(ax=ax[0], show=False)
raw_eeg_cropped_filtered.plot_psd(ax=ax[1], show=False)

ax[0].set_title('PSD before filtering')
ax[1].set_title('PSD after filtering')
ax[1].set_xlabel('Frequency (Hz)')
fig.set_tight_layout(True)
plt.show()

In [ ]:
raw.plot()

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>How many <strong>visual</strong> events are in the data?</li>
    </ul>
</div>

## Plot the raw data again, but add event markers

In [ ]:
raw.plot(events=events, event_id=event_id)

## Gather some info about the data

In [ ]:
raw.info

In [ ]:
raw.info['meas_date']

In [ ]:
raw.info['sfreq']

In [ ]:
raw.info['bads']

In [ ]:
raw.info['chs'][0]

## Visualize the sensor locations

In [ ]:
raw.plot_sensors(ch_type='eeg')

In [ ]:
raw.plot_sensors(kind='3d', ch_type='eeg')

## Mark channels as bad

Mark an additional EEG channel as bad and view the topoplot.

In [ ]:
raw.info['bads']

In [ ]:
raw.info['bads'] += ['EEG 051']
raw.plot_sensors(ch_type='eeg')

## Select only a subset of the channels

In [ ]:
raw_eeg = raw.copy().pick_types(meg=False, eeg=True, eog=True, exclude=[])
len(raw_eeg.ch_names)

In [ ]:
raw_eeg.info

In [ ]:
raw_eeg.plot(events=events, event_id=event_id)

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Select only MEG channels ("meg")</li>
         <li>Select only magnetometer channels ("mag")</li>
    </ul>
</div>

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Filter the raw data with a 1 Hz high-pass and a 30 Hz low-pass filter and plot the PSD.</li>
    </ul>
</div>

## Save the data

In [ ]:
raw_eeg_cropped_filtered.save(pathlib.Path('out_data') / 'eeg_cropped_filt_raw.fif', 
                              overwrite=True)